In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # matrix construction
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import json
import os

import spacy
import matplotlib
import plotly.express as px
import plotly.subplots as sp
from plotly.subplots import make_subplots

import sklearn.metrics
import numpy as np

In [2]:
nlp = spacy.load("en_core_web_sm", disable=['parser','tagger', 'parser', 'ner']) 

# Opening Files: 

In [54]:
dataset = "merged_dataset.json"
path_data =  "../src/merged_dataset.json"


with open(path_data) as file:
    open_data = json.load(file)

# Converting to Data Frames: 

df = pd.DataFrame(open_data).transpose().reset_index(drop = True)

# Feature Extraction:

In [16]:
# ------------ Count Vectorizer --------------- 

# cv = CountVectorizer(analyzer='word', stop_words='english', lowercase=False,
                        #min_df=0.01) # if words used less than 0.001 % --> ignore  
# data = cv.fit_transform(df_tech['text']) 

# df_dtm = pd.DataFrame(data.toarray(), columns=cv.get_feature_names())

# ---------------- TF-IDF ---------------------: 

tf_idf = TfidfVectorizer(analyzer = 'word', stop_words='english', lowercase=False,
                        min_df = 0.01)

data = tf_idf.fit_transform(df_tech['text'])

df_dtm = pd.DataFrame(data.toarray(), columns=tf_idf.get_feature_names())



/Users/liviafries/opt/anaconda3/envs/auto_cti/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
df_for_model = pd.concat(  # concatenate label 
    [df_dtm, df_tech[['tech_id']]], axis=1)

print(df_for_model.shape)

(7313, 33724)


In [19]:
df_for_model.head()

,00,000,0000,00000,000000,00000000,0000000000,0000000017,00000001,00000002,...,台灣,日本,日本語,香港,대한민국,한국어,ﬁle,ﬁles,ﬁrst,tech_id
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T1548
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T1071
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T1547
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T1543
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,T1555


In [21]:
df_agg = df_concat.groupby(['tech_name']).mean().reset_index()
tech_name = df_agg['tech_name'].unique()[0] # get first tech name

NameError: name 'df_concat' is not defined

In [20]:
df_agg.loc[df_agg['tech_name'] == tech_name].iloc[:,1:].transpose().sort_values(by = 0, ascending=False)

NameError: name 'df_agg' is not defined

In [44]:
#df_tech.loc[df_tech['tech_name'] =='/etc/passwd and /etc/shadow']

,url,mitre_domain,tactic_name,tech_name,tech_id,text
1317,https://www.tldp.org/LDP/lame/LAME/linux-admin...,[enterprise-attack],[credential-access],/etc/passwd and /etc/shadow,T1003.008,Linux Password & Shadow File FormatsLinux Admi...
1318,https://www.cyberciti.biz/faq/unix-linux-passw...,[enterprise-attack],[credential-access],/etc/passwd and /etc/shadow,T1003.008,Please Wait... | Cloudflare Please enable coo...


In [37]:
df_dtm.head() # haven't lemmatised yet ! 

,00,000,01,02,03,04,05,06,07,08,...,worldwide,worth,write,writing,written,www,xml,year,years,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,1,4,0,0,0,4,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,1,4,0,0,0,4,1,0


# Visualisation

In [22]:
# Visualise frequence of words:

def plot_freq_cats(df, top_n=15):
    """Plot the frequent words used in the different categories

    Parameters
    ----------
    df : pd.DataFrame
        Dataset to get the word frequency
    top_n : int, default = 15
        number of most frequent words to show
    """
    nb_cats = df["tech_name"].nunique() # number of tech_names
    fig = make_subplots(rows=nb_cats, cols=1, vertical_spacing=0.02,
                       subplot_titles=["Word frequency for "+ cat for cat in df["tech_name"].unique()])
    
    for i, cat in enumerate(df["tech_name"].unique()):
        df_freq = (df_agg.loc[df_agg['tech_name'] == cat]
                   .iloc[:,1:].transpose().reset_index())
                  
        df_freq.columns = ['words', 'frequency']
        df_freq = (df_freq.sort_values(by = 'frequency', ascending=False)) 

                  
        fig_site = px.bar(df_freq.iloc[:top_n,:],
                      x="words", y="frequency", orientation="v",
                      width=800, height=300, title=f"Word frequency for {cat} ")
        fig.append_trace(fig_site["data"][0], row=i+1, col=1)
    
    fig.update_layout(
    autosize=False,
    width=800,
    height=4400,)
    fig.show()


In [23]:
plot_freq_cats(df_agg.iloc[:10, :])

NameError: name 'df_agg' is not defined

# Model: 

In [24]:
# Train and test: First delete techniques less than 9 

# We fix the random state to have the same dataset in our different tests

x_train, x_test, y_train, y_test = train_test_split(df_for_model.drop(["tech_id"], axis=1),  # 70 training, 30 testing
                                                    df_for_model["tech_id"], test_size=0.3,
                                                    random_state = 10)

naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(x_train, y_train)

y_pred = naive_bayes_classifier.predict(x_test)

In [39]:
sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None) # calculate accuracy 

0.016666666666666666

In [40]:
# create a dataframe to understand why accuracy is so low: 

df_res = pd.DataFrame({'y_true': y_test, 'y_pred': y_pred})


In [41]:
df_res

,y_true,y_pred
1518,T0811,T1558
1246,T1614.001,T1546.004
544,T1588,T1558
1343,T1221,T1546.004
428,T1584,T1558
...,...,...
174,T1079,T1546.004
387,T1499,T1558
1561,T0848,T1558
846,T1183,T1558


In [50]:
index_usedWords = np.flatnonzero(df_dtm.iloc[0, :] > 0)

In [53]:
df_dtm.iloc[0, :][index_usedWords].index.tolist()

['11',
 '16',
 '2005',
 '2012',
 '2019',
 '2022',
 '37',
 '64',
 'ADMIN',
 'AM',
 'All',
 'Applications',
 'Attacks',
 'Aug',
 'Back',
 'Boot',
 'Broadcom',
 'Buy',
 'Chain',
 'Community',
 'Company',
 'Component',
 'Cookie',
 'Copies',
 'Copyright',
 'Creates',
 'DOMAIN',
 'Deletes',
 'Display',
 'Distributed',
 'Downloads',
 'Dropper',
 'Drops',
 'Endpoint',
 'Enter',
 'Enterprise',
 'Entries',
 'Events',
 'Executes',
 'FROM',
 'Figure',
 'Files',
 'Finally',
 'GET',
 'HTTP',
 'Higher',
 'Home',
 'Hosted',
 'How',
 'IP',
 'Image',
 'Inc',
 'Information',
 'It',
 'JPEG',
 'Johnson',
 'Keywords',
 'LLC',
 'Library',
 'Link',
 'Links',
 'Logic',
 'MBR',
 'Master',
 'Members',
 'NAME',
 'No',
 'Note',
 'Only',
 'Options',
 'Policy',
 'Powered',
 'Press',
 'Privacy',
 'Products',
 'Protection',
 'Recommend',
 'Record',
 'Register',
 'Related',
 'Reporter',
 'Reserved',
 'Resource',
 'Response',
 'Rights',
 'STATE',
 'Search',
 'Security',
 'Server',
 'Service',
 'Shamoon',
 'Shares',
 'Si

In [54]:
df_concat.iloc[0, -1]

'T1487'